## 간단한 예제로 Map-Reduce 구현하기

하둡, 스파크의 근간이 되는 개념인 맵리듀스(Map-Reduce)를 Naive한 파이썬으로 재구성해봄으로써, 그 구조를 배워보자.

Reference : [깃헙 - exercises-in-programming-style](https://github.com/crista/exercises-in-programming-style)

여기서 구현하고자 하는 프로그램은 매우 간단하다.

텍스트 파일이 제공되면 가장 빈도가 높은 단어 N(예를 듦녀 25)개와 그에 해당하는 빈도를 내림차순으로 출력한다. 대문자는 모두 소문자로 정규화하고 "the", "for" 등과 같은 의미 없는 단어는 무시해야 한다. 문제를 단순하게 만들기 위해 빈도가 같은 단어의 순서는 신경 쓰지 않는다. 이 계산 작업을 **단어 빈도**라고 한다.

아래 코드가 가장 단순한 형태의 맵리듀스 함수이다.

In [1]:
import sys, re, operator, string
from functools import reduce
from multiprocessing.pool import Pool

"""
맵 리듀스용 함수
"""
def partition(data_str, nlines):
    """
    Partition the input data_str (a big string)
    into chunks of nlines
    """
    lines = data_str.split('\n')
    for i in range(0, len(lines), nlines):
        yield '\n'.join(lines[i:i+nlines])
        
def split_words(data_str):
    """
    Takes a string, returns a list of pairs (word, 1), 
    one for each word in the input, so
    [(w1, 1), (w2, 2), ..., (wn, 1)]
    """
    def _scan(str_data):
        pattern = re.compile("[\W_]+") # 특수기호 외 모든 단어
        return pattern.sub(' ', str_data).lower().split()
    
    def _remove_stop_wrods(word_list):
        with open("./stop_words.txt", 'r') as f:
            stop_words = f.read().split(',')
        stop_words.extend(list(string.ascii_lowercase))
        return [w for w in word_list if not w in stop_words]
    
    # 입력 내용을 단어로 나누는 실제 작업
    result = []
    words = _remove_stop_wrods(_scan(data_str))
    for w in words:
        result.append((w, 1))
    return result
    
def count_words(pairs_list_1, pairs_list_2):
    """
    Takes two lists of pairs of the 
    [(W1, 1), ...]
    and returns a list of pairs [(w1, frequency), ...],
    where frequency is the sum of all the reported occurrences
    """
    mapping = { k : v for k, v in pairs_list_1}
    for p in pairs_list_2:
        if p[0] in mapping:
            mapping[p[0]] += p[1]
        else:
            mapping[p[0]] = 1
    return mapping.items()

"""
보조 함수
"""
def read_file(path_to_file):
    with open(path_to_file) as f:
        data = f.read()
    return data

def sort(word_freq):
    return sorted(word_freq, key=operator.itemgetter(1), reverse=True)

"""
주함수
"""
splits = map(split_words, 
             partition(
                 read_file(
                     "./pride-and-prejudice.txt"),
                     200))

word_freq = sort(reduce(count_words,splits,[]))
print(word_freq[:20])

[('mr', 786), ('elizabeth', 635), ('very', 488), ('darcy', 418), ('such', 395), ('mrs', 343), ('much', 329), ('more', 327), ('bennet', 323), ('bingley', 306), ('jane', 295), ('miss', 283), ('one', 275), ('know', 239), ('before', 229), ('herself', 227), ('though', 226), ('well', 224), ('never', 220), ('sister', 218)]


이 형식에서는 문제의 입력 데이터를 덩어리로 분할하고 각 덩어리를 서로 독립적으로, 어쩌면 병렬로 처리하며, 마지막에는 그 결과를 재결합한다. 맵리듀스(Map Reduce) 형식은 두 가지 주요 추상화로 이뤄진다. 

1. 맵 함수는 데이터 덩어리와 함수를 인자로 받아 그 함수를 각 덩어리에 독집적으로 적용해 결과 모음을 만들어 낸다.
2. 리듀스 함수는 결과 모음과 함수를 인자로 취해 그 모음에서 어떤 전역 정보(global knowledge)를 추출하기 위해 그 함수를 결과 모음에 적용한다.

작업 순서는 

**글 전체** -> `partition` -> **데이터 덩어리** -> `split_words` -> **결과 모음** ->`count_words` -> **단어 빈도**로 진행된다.

*  `partition` 함수:  여러 줄로 구성된 문자열과 줄 수를 입력 받아 요청한 줄 수 만큼 문자열을 생성하는 생성기 역할
* `split_words` 함수:  여러 줄로 구성된 문자열 하나, 위에서 사용한 것처럼 200줄 단위인 구역 하나를 받아 처리한다. 
* `count_words` 함수: reduce의 첫번째 인자로 사용된 작업 함수로, 리스트를 병합하면서 총 단어의 갯수를 뽑아내는 데에 이용된다. 

오만과 편견의 글은 13,426줄 정도로 구성되어 있다. 이를 200줄 단위로 구역 68개로 분할한다.(partition). 그리고 이 데이터 덩어리에서 각각 단어 수를 세게 된다.(split_words). split_words 이후에 나온 결과물들을 한 데 모아 수합함으로써, 일을 마치게 된다 (count_words)

-------


### 시스템 설계 관점에서 본 이러한 형식

맵 리듀스는 데이터를 분할하고 독립적으로 처리할 수 있으며, 마지막에는 부분적인 결과를 재결합할 수 있는 데이터 집약적인 응용 프로그램에 자연스레 어울린다.

### 위 설계에서의 한계 

여기에서 문제는 바로 Reduce에 있다. Map 부분은 병렬로 처리하지만, Reduce는 결국 단일한 프로세스에서 작업하게 된다. 그렇기 때문에 마지막 부분이 병목현상을 발생하게 되어, 위의 방식으로 병렬 처리하더라도 큰 이익을 얻지 못하게 된다. Reduce마저도 병렬 처리할 수 없을까? -> 이런 것들을 해결한 것이 바로 하둡. 

다음 시간에 좀 더 공부같이 해보아요!